# Ceneo Scraper

## Import bibliotek


In [11]:
import os
import json
import requests
from bs4 import BeautifulSoup

In [2]:
product_id = "108481122"
product_id = input("podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

Wysłanie do serwera Ceneo.pl rządania dostępu do strony z opiniami o produkcie.

In [3]:
response = requests.get(url)

rządanie wysyłamy tylko raz, a potem pracujemy już na zmiennej response, żeby nas ceneo nie zbanowało, że dużo requestów im wysyłamy

In [4]:
response.status_code #

200

jeśli damy jakąś zmienną na końcu to jupier wyświetli nam jej wartość

status code 200 oznacza, że się udało

## Przekształcenie kodu html w postacii tekstowej do obiektu reprezentującego strukturę DOM i wydobycie ze struktury DOM znaczników odpowiadających opiniom konsumentów

In [6]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review") #ten zwróci nam wszystkie pasujące opinie
#opinion = page.select_one("div.js_product-review") #ten zwróci nam tylko jedną pasującą opinię (tę pierwszą napotkaną)
#print(page)
print(type(page))
print(type(opinions))
#print(type(opinion))

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>


## Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii  i jej składowych

In [13]:
def extract(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except: TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except: AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

SyntaxError: invalid syntax (1997104998.py, line 10)

In [ ]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        key: extract(opinion, *value)
            for key, value in selectors.items()
    }
    all_opinions.append(single_opinion)

In [ ]:
all_opinions = []
while():
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product_review")
    for opinion in opinions:
        single_opinion = {
        key: extract(opinion, *value)
            for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", "href")
    except: TypeError:
        url = None
        

    

In [ ]:
selectors = {
    "opinion_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recommend": ("span.user-post__author-recomendation> em",),
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    "pros": ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    "opinion_date": ("span.user-post__published > time:nth-child(1)", "datetime"),
    "purchase_date": ("span.user-post__published > time:nth-child(2)","datetime"),
    "up_vote": ("button.vote-yes","data-total-vote"),
    "down_vote": ("button.vote-no","data-total-vote")
}

tutaj robimy list comprehension czy jakoś tak to się nazywa:

In [8]:
single_opinion = {
    key: extract(opinion, *value) #gwiazdka wypakowue nam listę/krotkę i wyyciąga poszczególne elementy (cokolwiek to znaczy)
        for key, value in selectors.items() = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
}

SyntaxError: invalid syntax (3597983064.py, line 3)

In [ ]:
print(type(*selectors['opinion_date'])) #dictionary comprehension

In [30]:
single_opinion = {
    "opinion_id": extract(opinion, None, "data-entry-id"),
    "author": extract(opinion, "span.user-post__author-name"),
    "recommend": extract(opinion, "span.user-post__author-recomendation> em"),
    "stars": extract(opinion, "span.user-post__score-count"),
    "content": extract(opinion, "div.user-post__text"),
    "cons": extract(opinion, "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    "pros": extract(opinion, "div.review-feature__title--positives ~ div.review-feature__item", None, True),
    "opinion_date": extract(opinion, "span.user-post__published > time:nth-child(1)", "datetime"),
    "purchase_date": extract(opinion,"span.user-post__published > time:nth-child(2)","datetime"),
    "up_vote": extract(opinion,"button.vote-yes","data-total-vote"),
    "down_vote": extract(opinion,"button.vote-no","data-total-vote")
}

Zapis opinii o produkcie do pliku json do katalogu opinions

In [12]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json', "w", encoding="UTF-8") as jf: #jf jak json file
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
#all_opinions

NameError: name 'all_opinions' is not defined

## Struktura opinii w serwisie Ceneo.pl

|składowa|selektor|zmienna|
|---|---|---|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation> em |recommend|
|liczba gwiazdek|span.user-post__score-count|stars|
|treść opinii|div.user-post__text|content|
|lista wad|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|lista zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|


https://www.ceneo.pl/108481122#tab=reviews


In [6]:
response.text

'\r\n\r\n<!DOCTYPE html>\r\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 8]>         <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if gt IE 8]><!-->\r\n<html class="no-js  non-direct ab-inactive" lang="pl">\r\n<!--<![endif]-->\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <link rel="preconnect" href="https://image.ceneo.pl">\n<link rel="preconnect" href="https://image.ceneostatic.pl">\n<link rel="preconnect" href="https://www.google-analytics.com">\n<link rel="preconnect" href="https://www.googleadservices.com">\n<link rel="preconnect" href="https://adservice.google.com">\n<link rel="preconnect" href="https://www.google.pl">\n<link rel="preconnect" href="https://www.google.com">\n<link rel="preconnect" 